In [1]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
import mne
import numpy as np
import os
import pickle

In [3]:
def sample_process(root_folder, k, N, epoch_sec, train_test_val, index):
    for i, j in enumerate(index):
        if i % N == k:
            if k == 0:
                print ('Progress: {} / {}'.format(i, len(index)))

            # load signal "X" part
            data = mne.io.read_raw_edf(root_folder + '/' + list(filter(lambda x: (x[:6] == j) and ('PSG' in x), os.listdir(root_folder)))[0])
            X = data.get_data()[:2, :]
            
            # load label "Y" part
            ann = mne.read_annotations(root_folder + '/' + list(filter(lambda x: (x[:6] == j) and ('Hypnogram' in x), os.listdir(root_folder)))[0])
            labels = []
            for dur, des in zip(ann.duration, ann.description):
                for i in range(int(dur) // 30):
                    labels.append(des[-1])

            # slice the EEG signals into non-overlapping windows, window size = sampling rate per second * second time = 100 * windowsize
            for slice_index in range(X.shape[1] // (100 * epoch_sec)):
                # ingore the no labels
                if labels[slice_index] == '?':
                    continue
                path = './SLEEP_data/cassette_processed/{}/'.format(train_test_val) + 'cassette-' + j + '-' + str(slice_index) + '.pkl'
                pickle.dump({'X': X[:, slice_index * 100 * epoch_sec: (slice_index+1) * 100 * epoch_sec], \
                    'y': labels[slice_index]}, open(path, 'wb'))


In [4]:
from pyedflib import highlevel
import numpy as np
import glob

path1 = '/Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-cassette'
path2 = '/Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-telemetry'
data_list_cassette = sorted(glob.glob(path1+'/**'))
data_list_telemetry = sorted(glob.glob(path2+'/**')) 

print('number of cattette files',len(data_list_cassette))
print('number of telemetry files',len(data_list_telemetry))


number of cattette files 306
number of telemetry files 88


In [5]:
#First, investigate cassette files.
trains_SC = [x for x in data_list_cassette if x.endswith('PSG.edf')]
labels_SC= [x for x in data_list_cassette if x.endswith('Hypnogram.edf')]

trains_ST = [x for x in data_list_telemetry if x.endswith('PSG.edf')]
labels_ST= [x for x in data_list_telemetry if x.endswith('Hypnogram.edf')]

print('number of SC_train samples:',len(trains_SC))
print('number of SC_labels:',len(labels_SC))
print('number of ST_train samples:',len(trains_ST))
print('number of ST_labels:',len(labels_ST))

number of SC_train samples: 153
number of SC_labels: 153
number of ST_train samples: 44
number of ST_labels: 44


In [6]:
for i in range(len(trains_SC)):
    print(trains_SC[i].split('/')[-1].split('-')[0])
    print(labels_SC[i].split('/')[-1].split('-')[0])
    print('-----')

SC4001E0
SC4001EC
-----
SC4002E0
SC4002EC
-----
SC4011E0
SC4011EH
-----
SC4012E0
SC4012EC
-----
SC4021E0
SC4021EH
-----
SC4022E0
SC4022EJ
-----
SC4031E0
SC4031EC
-----
SC4032E0
SC4032EP
-----
SC4041E0
SC4041EC
-----
SC4042E0
SC4042EC
-----
SC4051E0
SC4051EC
-----
SC4052E0
SC4052EC
-----
SC4061E0
SC4061EC
-----
SC4062E0
SC4062EC
-----
SC4071E0
SC4071EC
-----
SC4072E0
SC4072EH
-----
SC4081E0
SC4081EC
-----
SC4082E0
SC4082EP
-----
SC4091E0
SC4091EC
-----
SC4092E0
SC4092EC
-----
SC4101E0
SC4101EC
-----
SC4102E0
SC4102EC
-----
SC4111E0
SC4111EC
-----
SC4112E0
SC4112EC
-----
SC4121E0
SC4121EC
-----
SC4122E0
SC4122EV
-----
SC4131E0
SC4131EC
-----
SC4141E0
SC4141EU
-----
SC4142E0
SC4142EU
-----
SC4151E0
SC4151EC
-----
SC4152E0
SC4152EC
-----
SC4161E0
SC4161EC
-----
SC4162E0
SC4162EC
-----
SC4171E0
SC4171EU
-----
SC4172E0
SC4172EC
-----
SC4181E0
SC4181EC
-----
SC4182E0
SC4182EC
-----
SC4191E0
SC4191EP
-----
SC4192E0
SC4192EV
-----
SC4201E0
SC4201EC
-----
SC4202E0
SC4202EC
-----
SC4211E0
SC4211E

label과 data들이 순서쌍을 이루게 끔 list에 담겨있는 것을 확인 할 수 있음

In [6]:
data = mne.io.read_raw_edf(trains_SC[0])
X = data.get_data()[:2,:]

Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [7]:
X.shape # EEG signal

(2, 7950000)

In [8]:
ann = mne.read_annotations(labels_SC[0])
print(ann.duration)
print('------')
print(ann.description)

[3.063e+04 1.200e+02 3.900e+02 3.000e+01 3.000e+01 1.500e+02 3.000e+01
 6.000e+01 6.000e+01 3.000e+01 1.200e+02 3.000e+01 1.200e+02 3.000e+01
 6.000e+01 6.000e+01 1.200e+02 3.000e+01 3.000e+01 1.200e+02 2.100e+02
 3.000e+01 6.000e+01 1.200e+02 3.000e+01 1.500e+02 6.000e+01 3.000e+01
 3.000e+01 3.000e+01 1.200e+02 1.500e+02 3.000e+01 3.000e+01 6.000e+01
 3.000e+01 9.000e+01 1.890e+03 3.000e+01 2.100e+02 6.000e+01 9.000e+01
 1.500e+02 3.000e+01 8.700e+02 1.800e+02 2.400e+02 3.000e+01 1.200e+02
 9.000e+01 3.000e+01 9.300e+02 3.000e+01 3.000e+01 6.000e+01 3.000e+01
 3.000e+01 3.000e+01 3.000e+01 3.000e+01 3.000e+01 1.500e+02 6.000e+01
 9.000e+01 6.000e+01 3.000e+01 3.000e+01 6.000e+01 6.000e+01 1.800e+02
 6.000e+01 3.000e+01 3.000e+01 2.700e+02 3.000e+01 6.000e+01 3.000e+01
 2.100e+02 3.000e+01 6.000e+01 3.000e+01 1.800e+02 8.700e+02 3.000e+01
 6.000e+01 3.000e+01 1.200e+02 8.700e+02 3.000e+01 3.000e+01 3.000e+01
 3.000e+02 3.000e+01 3.900e+02 3.000e+01 3.000e+01 3.000e+01 3.000e+01
 1.800

In [9]:
labels = []
for dur, des in zip(ann.duration, ann.description):
    for i in range(int(dur) // 30):
        labels.append(des[-1])

In [10]:
sum(ann.duration)*100


8640000.0

In [13]:
print(len(labels))
print(7950000//(100*30))

2880
2650


In [14]:
header = highlevel.read_edf(labels_SC[0])[2]['annotations']
print(header)

[[0.0, 30630.0, 'Sleep stage W'], [30630.0, 120.0, 'Sleep stage 1'], [30750.0, 390.0, 'Sleep stage 2'], [31140.0, 30.0, 'Sleep stage 3'], [31170.0, 30.0, 'Sleep stage 2'], [31200.0, 150.0, 'Sleep stage 3'], [31350.0, 30.0, 'Sleep stage 4'], [31380.0, 60.0, 'Sleep stage 3'], [31440.0, 60.0, 'Sleep stage 4'], [31500.0, 30.0, 'Sleep stage 3'], [31530.0, 120.0, 'Sleep stage 4'], [31650.0, 30.0, 'Sleep stage 3'], [31680.0, 120.0, 'Sleep stage 4'], [31800.0, 30.0, 'Sleep stage W'], [31830.0, 60.0, 'Sleep stage 3'], [31890.0, 60.0, 'Sleep stage 2'], [31950.0, 120.0, 'Sleep stage 3'], [32070.0, 30.0, 'Sleep stage 4'], [32100.0, 30.0, 'Sleep stage 3'], [32130.0, 120.0, 'Sleep stage 4'], [32250.0, 210.0, 'Sleep stage 2'], [32460.0, 30.0, 'Sleep stage 3'], [32490.0, 60.0, 'Sleep stage 2'], [32550.0, 120.0, 'Sleep stage 3'], [32670.0, 30.0, 'Sleep stage 1'], [32700.0, 150.0, 'Sleep stage 2'], [32850.0, 60.0, 'Sleep stage 3'], [32910.0, 30.0, 'Sleep stage 2'], [32940.0, 30.0, 'Sleep stage 3'], [329

duration이 0초부터 시작.
끝에 잔여 시긴호가 있는 걸로 확인

In [15]:
for slice_index in range(X.shape[1] // (100 * 30)):
    # ingore the no labels
    if labels[slice_index] == '?':
        continue
    print(X[:, slice_index * 100 * 30: (slice_index+1) * 100 * 30].shape)
    print(labels[slice_index])
    if slice_index >= 10:
        break

(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W
(2, 3000)
W


annotaion과 그에 해당하는 신호들을 추출 가능!

----

다양한 dataset을 가지고 SSL을 하고자 하는데, sampling rate를 200Hz로 통일 하고자 함.
sampling rate가 200Hz가 정답이 아님. 데이터를 불러올때 마다 원하는 sampling rate로 불러 올 수 있는데(mne.fikter.resample()), 이 과정이 오래걸려서 애초에 200Hz로 저장하려고 함.

In [16]:
X = data.get_data()[:2,:]
X1 = mne.filter.resample(X, up = 2.0) # upsampling to 200Hz  
print(X.shape)
print(X1.shape[1]/2)

(2, 7950000)
7950000.0


In [17]:
import os
for i in range(len(trains_SC)):
    print(i)
    subjetc_id = trains_SC[i].split('/')[-1].split('-')[0]
    
    # load signal and upsampling
    data = mne.io.read_raw_edf(trains_SC[i])
    data = data.get_data()[:2,:]
    data = mne.filter.resample(data, up = 2.0) # upsampling to 200Hz  
    
    # load annotaion
    ann = mne.read_annotations(labels_SC[i])
    labels = []
    for dur, des in zip(ann.duration, ann.description):
        for i in range(int(dur) // 30):
            labels.append(des[-1])

    for slice_index in range(data.shape[1] // (200 * 30)): # 200Hz, 30epochs
    # ingore the no labels
        if labels[slice_index] == '?':
            continue
            
        
        adress = '/Users/josangmin/EEG/Sleep_edfx_30s_200Hz/SC'
        if not os.path.exists(adress+'/data/{}'.format(subjetc_id)):
            os.makedirs(adress +'/data/{}'.format(subjetc_id))
            data_adress = adress +'/data/{}'.format(subjetc_id)
            os.makedirs(adress + '/label/{}'.format(subjetc_id))
            label_adress = adress +'/label/{}'.format(subjetc_id)
            
        data_path = data_adress +'/data'+ str(subjetc_id) + '_' + str(slice_index) + '.pkl'
        label_path = label_adress + '/label' + str(subjetc_id) + '_' + str(slice_index) + '.pkl'
        X = data[:, slice_index * 200 * 30: (slice_index+1) * 200 * 30]
        with open(file = data_path, mode='wb') as f:
            pickle.dump(X, f)
        with open(file = label_path, mode='wb') as f:
            pickle.dump(labels[slice_index], f)
        
        

0
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
1
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4002E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
2
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4011E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
3
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4012E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
4
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-cassette/SC4021E0-PSG.edf...
EDF file detected
Setti

In [18]:
data_path = '/Users/josangmin/EEG/Sleep_edfx_30s_200Hz/SC/data/SC4001E0/' + 'dataSC4001E0_0.pkl'
with open(data_path, 'rb') as f:
    a = pickle.load(f)
a.shape

(2, 6000)

In [19]:
for i in range(len(trains_ST)):
    print(i)
    subjetc_id = trains_ST[i].split('/')[-1].split('-')[0]
    
    # load signal and upsampling
    data = mne.io.read_raw_edf(trains_ST[i])
    data = data.get_data()[:2,:]
    data = mne.filter.resample(data, up = 2.0) # upsampling to 200Hz  
    
    # load annotaion
    ann = mne.read_annotations(labels_ST[i])
    labels = []
    for dur, des in zip(ann.duration, ann.description):
        for i in range(int(dur) // 30):
            labels.append(des[-1])
    
    for slice_index in range(len(labels)): # 200Hz, 30epochs
    # ingore the no labels
        if labels[slice_index] == '?':
            continue
     
        adress = '/Users/josangmin/EEG/Sleep_edfx_30s_200Hz/ST'
        if not os.path.exists(adress+'/data/{}'.format(subjetc_id)):
            os.makedirs(adress +'/data/{}'.format(subjetc_id))
            data_adress = adress +'/data/{}'.format(subjetc_id)
            os.makedirs(adress + '/label/{}'.format(subjetc_id))
            label_adress = adress +'/label/{}'.format(subjetc_id)
            
        data_path = data_adress +'/data'+ str(subjetc_id) + '_' + str(slice_index) + '.pkl'
        label_path = label_adress + '/label' + str(subjetc_id) + '_' + str(slice_index) + '.pkl'
        X = data[:, slice_index * 200 * 30: (slice_index+1) * 200 * 30]
        with open(file = data_path, mode='wb') as f:
            pickle.dump(X, f)
        with open(file = label_path, mode='wb') as f:
            pickle.dump(labels[slice_index], f)
        
        

0
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-telemetry/ST7011J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
1
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-telemetry/ST7012J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
2
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-telemetry/ST7021J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
3
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-telemetry/ST7022J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
4
Extracting EDF parameters from /Users/josangmin/Downloads/sleep-edf-database-expanded-1.0.0/sleep-telemetry/ST7041J0-PSG.edf...
EDF file detected


In [39]:
header = highlevel.read_edf(labels_ST[0])[2]['annotations']
print(header)

[[0.0, 1560.0, 'Sleep stage W'], [1560.0, 90.0, 'Sleep stage 1'], [1650.0, 570.0, 'Sleep stage 2'], [2220.0, 120.0, 'Sleep stage 1'], [2340.0, 60.0, 'Sleep stage 2'], [2400.0, 60.0, 'Sleep stage 3'], [2460.0, 30.0, 'Sleep stage 1'], [2490.0, 150.0, 'Sleep stage 2'], [2640.0, 90.0, 'Sleep stage 3'], [2730.0, 30.0, 'Sleep stage 2'], [2760.0, 30.0, 'Sleep stage 1'], [2790.0, 90.0, 'Sleep stage 2'], [2880.0, 120.0, 'Sleep stage W'], [3000.0, 60.0, 'Sleep stage 1'], [3060.0, 240.0, 'Sleep stage 2'], [3300.0, 240.0, 'Sleep stage W'], [3540.0, 60.0, 'Sleep stage 2'], [3600.0, 30.0, 'Sleep stage 3'], [3630.0, 30.0, 'Sleep stage 2'], [3660.0, 60.0, 'Sleep stage 1'], [3720.0, 90.0, 'Sleep stage W'], [3810.0, 120.0, 'Sleep stage 1'], [3930.0, 30.0, 'Sleep stage 2'], [3960.0, 60.0, 'Sleep stage 1'], [4020.0, 60.0, 'Sleep stage 2'], [4080.0, 90.0, 'Sleep stage 1'], [4170.0, 90.0, 'Sleep stage W'], [4260.0, 30.0, 'Sleep stage 1'], [4290.0, 150.0, 'Sleep stage 2'], [4440.0, 30.0, 'Sleep stage 3'], [4